##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 그라디언트 및 자동 미분 소개

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/guide/autodiff"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org에서보기</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/autodiff.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab에서 실행</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/guide/autodiff.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub에서 소스보기</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/guide/autodiff.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">노트북 다운로드</a></td>
</table>

## 자동 미분 및 기울기

[자동 미분](https://en.wikipedia.org/wiki/Automatic_differentiation) 은 신경망 훈련을위한 [역전 파와](https://en.wikipedia.org/wiki/Backpropagation) 같은 기계 학습 알고리즘을 구현하는 데 유용합니다.

мега гайд

## 설정

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf

## 그라디언트 계산

자동으로 구별하기 위해 TensorFlow는 *전달* 단계에서 어떤 작업이 어떤 순서로 발생하는지 기억해야합니다. 그런 다음 *역방향 패스* 동안 TensorFlow는이 작업 목록을 역순으로 탐색하여 기울기를 계산합니다.

## 그라데이션 테이프

TensorFlow는 자동 차별화를 위해 [tf.GradientTape](https://www.tensorflow.org/api_docs/python/tf/GradientTape) API를 제공합니다. 즉, 일반적으로 `tf.Variable` s와 같은 일부 입력에 대한 계산의 기울기를 계산합니다. TensorFlow는 `tf.GradientTape` 의 컨텍스트 내에서 실행 된 관련 작업을 "테이프"에 "기록"합니다. 그런 다음 TensorFlow는 해당 테이프를 사용하여 [역방향 모드 미분을](https://en.wikipedia.org/wiki/Automatic_differentiation) 사용하여 "기록 된"계산의 기울기를 계산합니다.

다음은 간단한 예입니다.

In [ ]:
x = tf.Variable(3.0)

with tf.GradientTape() as tape:
  y = x**2

일부 작업을 기록한 후에는 `GradientTape.gradient(target, sources)` 를 사용하여 일부 소스 (종종 모델의 변수)와 관련된 일부 타겟 (종종 손실)의 기울기를 계산합니다.

In [ ]:
# dy = 2x * dx
dy_dx = tape.gradient(y, x)
dy_dx.numpy()

위의 예제는 스칼라를 사용하지만 `tf.GradientTape` 는 모든 텐서에서 쉽게 작동합니다.

In [ ]:
w = tf.Variable(tf.random.normal((3, 2)), name='w')
b = tf.Variable(tf.zeros(2, dtype=tf.float32), name='b')
x = [[1., 2., 3.]]

with tf.GradientTape(persistent=True) as tape:
  y = x @ w + b
  loss = tf.reduce_mean(y**2)

두 변수에 대한 `y` 의 기울기를 얻으려면 두 변수를 모두 소스로 `gradient` 메서드에 전달할 수 있습니다. 테이프는 소스가 전달되는 방식에 대해 유연하며 목록 또는 사전의 중첩 된 조합을 허용하고 동일한 방식으로 구조화 된 그래디언트를 반환합니다 ( `tf.nest` 참조).

In [ ]:
[dl_dw, dl_db] = tape.gradient(loss, [w, b])

각 소스에 대한 그라디언트는 소스의 모양을 갖습니다.

In [ ]:
print(w.shape)
print(dl_dw.shape)

다음은 다시 기울기 계산이며, 이번에는 변수 사전을 전달합니다.

In [ ]:
my_vars = {
    'w': tf.Variable(tf.random.normal((3, 2)), name='w'),
    'b': tf.Variable(tf.zeros(2, dtype=tf.float32), name='b')
}

grad = tape.gradient(loss, my_vars)
grad['b']

## 모델에 대한 기울기

[체크 포인트](checkpoint.ipynb) 및 [내보내기](saved_model.ipynb) 를 위해 `tf.Variables` 를 `tf.Module` 또는 하위 클래스 중 하나 ( `layers.Layer` , `keras.Model` )로 수집하는 것이 일반적입니다.

대부분의 경우 모델의 학습 가능한 변수와 관련하여 기울기를 계산하려고합니다. 모든 서브 클래스 때문에 `tf.Module` 자신의 변수를 집계 `Module.trainable_variables` 특성, 당신은 몇 줄의 코드에서 이러한 구배를 계산할 수 있습니다 : 

In [ ]:
layer = tf.keras.layers.Dense(2, activation='relu')
x = tf.constant([[1., 2., 3.]])

with tf.GradientTape() as tape:
  # Forward pass
  y = layer(x)
  loss = tf.reduce_mean(y**2)

# Calculate gradients with respect to every trainable variable
grad = tape.gradient(loss, layer.trainable_variables)

In [ ]:
for var, g in zip(layer.trainable_variables, grad):
  print(f'{var.name}, shape: {g.shape}')

<a id="watches"></a>

## 테이프가 감시하는 내용 제어

The default behavior is to record all operations after accessing a trainable `tf.Variable`. The reasons for this are:

- 테이프는 역방향 패스의 기울기를 계산하기 위해 정방향 패스에 기록 할 작업을 알아야합니다.
- 테이프는 중간 출력에 대한 참조를 보유하므로 불필요한 작업을 기록하고 싶지 않습니다.
- 가장 일반적인 사용 사례는 모든 모델의 학습 가능한 변수와 관련하여 손실의 기울기를 계산하는 것입니다.

예를 들어 다음은 `tf.Tensor` 가 기본적으로 "감시"되지 않고 `tf.Variable` 이 학습 가능하지 않기 때문에 기울기 계산에 실패합니다.

In [ ]:
# A trainable variable
x0 = tf.Variable(3.0, name='x0')
# Not trainable
x1 = tf.Variable(3.0, name='x1', trainable=False)
# Not a Variable: A variable + tensor returns a tensor.
x2 = tf.Variable(2.0, name='x2') + 1.0
# Not a variable
x3 = tf.constant(3.0, name='x3')

with tf.GradientTape() as tape:
  y = (x0**2) + (x1**2) + (x2**2)

grad = tape.gradient(y, [x0, x1, x2, x3])

for g in grad:
  print(g)

`GradientTape.watched_variables` 메서드를 사용하여 테이프에서 감시중인 변수를 나열 할 수 있습니다.

In [ ]:
[var.name for var in tape.watched_variables()]

`tf.GradientTape` 는 사용자가 시청 여부를 제어 할 수있는 후크를 제공합니다.

`tf.Tensor` 대한 그라디언트를 기록하려면 `GradientTape.watch(x)` 를 호출해야합니다.

In [ ]:
x = tf.constant(3.0)
with tf.GradientTape() as tape:
  tape.watch(x)
  y = x**2

# dy = 2x * dx
dy_dx = tape.gradient(y, x)
print(dy_dx.numpy())

반대로 모든 `tf.Variables` 감시의 기본 동작을 비활성화하려면 그래디언트 테이프를 만들 때 `watch_accessed_variables=False` 설정합니다. 이 계산은 두 개의 변수를 사용하지만 변수 중 하나에 대한 기울기 만 연결합니다.

In [ ]:
x0 = tf.Variable(0.0)
x1 = tf.Variable(10.0)

with tf.GradientTape(watch_accessed_variables=False) as tape:
  tape.watch(x1)
  y0 = tf.math.sin(x0)
  y1 = tf.nn.softplus(x1)
  y = y0 + y1
  ys = tf.reduce_sum(y)

`GradientTape.watch` 가 `x0` 에서 호출되지 않았으므로 이에 대해 그라디언트가 계산되지 않습니다.

In [ ]:
# dy = 2x * dx
grad = tape.gradient(ys, {'x0': x0, 'x1': x1})

print('dy/dx0:', grad['x0'])
print('dy/dx1:', grad['x1'].numpy())

## 중간 결과

`tf.GradientTape` 컨텍스트 내에서 계산 된 중간 값과 관련하여 출력의 기울기를 요청할 수도 있습니다.

In [ ]:
x = tf.constant(3.0)

with tf.GradientTape() as tape:
  tape.watch(x)
  y = x * x
  z = y * y

# Use the tape to compute the gradient of z with respect to the
# intermediate value y.
# dz_dx = 2 * y, where y = x ** 2
print(tape.gradient(z, y).numpy())

기본적으로 `GradientTape` 가 보유한 리소스는 `GradientTape.gradient()` 메서드가 호출되는 즉시 해제됩니다. 동일한 계산에 대해 여러 그라디언트를 계산하려면 `persistent` 그라디언트 테이프를 만듭니다. 이렇게하면 테이프 객체가 가비지 수집 될 때 리소스가 해제 될 때 `gradient()` 메서드를 여러 번 호출 할 수 있습니다. 예를 들면 :

In [ ]:
x = tf.constant([1, 3.0])
with tf.GradientTape(persistent=True) as tape:
  tape.watch(x)
  y = x * x
  z = y * y

print(tape.gradient(z, x).numpy())  # 108.0 (4 * x**3 at x = 3)
print(tape.gradient(y, x).numpy())  # 6.0 (2 * x)

In [ ]:
del tape   # Drop the reference to the tape

## 성능에 대한 참고 사항

- 그래디언트 테이프 컨텍스트 내에서 작업을 수행하는 것과 관련된 약간의 오버 헤드가 있습니다. 대부분의 빠른 실행에는 눈에 띄는 비용이 들지 않지만 필요한 영역에서만 테이프 컨텍스트를 사용해야합니다.

- 그래디언트 테이프는 메모리를 사용하여 입력 및 출력을 포함한 중간 결과를 저장하여 역방향 패스 중에 사용합니다.

    효율성을 위해 일부 작업 (예 : `ReLU` )은 중간 결과를 유지할 필요가 없으며 포워드 패스 중에 정리됩니다. 그러나 테이프에서 `persistent=True` 를 사용하면 *아무것도 버리지 않고* 최대 메모리 사용량이 더 높아집니다.

## 스칼라가 아닌 대상의 기울기

그래디언트는 기본적으로 스칼라에 대한 작업입니다.

In [ ]:
x = tf.Variable(2.0)
with tf.GradientTape(persistent=True) as tape:
  y0 = x**2
  y1 = 1 / x

print(tape.gradient(y0, x).numpy())
print(tape.gradient(y1, x).numpy())

따라서 여러 대상의 그라디언트를 요청하면 각 소스의 결과는 다음과 같습니다.

- 목표 합계의 기울기 또는 동등하게
- 각 대상의 기울기 합계입니다.

In [ ]:
x = tf.Variable(2.0)
with tf.GradientTape() as tape:
  y0 = x**2
  y1 = 1 / x

print(tape.gradient({'y0': y0, 'y1': y1}, x).numpy())

마찬가지로 목표가 스칼라가 아니면 합계의 기울기가 계산됩니다.

In [ ]:
x = tf.Variable(2.)

with tf.GradientTape() as tape:
  y = x * [3., 4.]

print(tape.gradient(y, x).numpy())

이렇게하면 손실 모음 합계의 기울기 또는 요소 별 손실 계산 합계의 기울기를 쉽게 얻을 수 있습니다.

각 항목에 대해 별도의 그라데이션이 필요한 경우 [Jacobians를](advanced_autodiff.ipynb#jacobians) 참조하십시오.

어떤 경우에는 Jacobian을 건너 뛸 수 있습니다. 요소 별 계산의 경우 각 요소가 독립적이므로 합계의 기울기는 입력 요소와 관련하여 각 요소의 미분을 제공합니다.

In [ ]:
x = tf.linspace(-10.0, 10.0, 200+1)

with tf.GradientTape() as tape:
  tape.watch(x)
  y = tf.nn.sigmoid(x)

dy_dx = tape.gradient(y, x)

In [ ]:
plt.plot(x, y, label='y')
plt.plot(x, dy_dx, label='dy/dx')
plt.legend()
_ = plt.xlabel('x')

## 제어 흐름

테이프는 실행되는 작업을 기록하기 때문에 Python 제어 흐름 (예 : `if` s 및 `while` s 사용)이 자연스럽게 처리됩니다.

ПРИВЕТ `if`. пока:

In [ ]:
x = tf.constant(1.0)

v0 = tf.Variable(2.0)
v1 = tf.Variable(2.0)

with tf.GradientTape(persistent=True) as tape:
  tape.watch(x)
  if x > 0.0:
    result = v0
  else:
    result = v1**2 

dv0, dv1 = tape.gradient(result, [v0, v1])

print(dv0)
print(dv1)

제어문 자체는 구별 할 수 없으므로 그래디언트 기반 옵티 마이저에게는 보이지 않습니다.

위의 예에서 `x` 값에 따라 테이프는 `result = v0` 또는 `result = v1**2` 합니다. `x` 대한 기울기는 항상 `None` 입니다.

In [ ]:
dx = tape.gradient(result, x)

print(dx)

## `None` 그라디언트 얻기

대상이 소스에 연결되어 있지 않으면 `None` 의 그라디언트가 표시됩니다.


In [ ]:
x = tf.Variable(2.)
y = tf.Variable(3.)

with tf.GradientTape() as tape:
  z = y * y
print(tape.gradient(z, x))

여기서 `z` 는 분명히 `x` 연결되어 있지 않지만 그래디언트를 분리 할 수있는 몇 가지 덜 분명한 방법이 있습니다.

### 1. 변수를 텐서로 대체했습니다.

ЙОу ["controlling what the tape watches"](#watches) you saw that the tape will automatically watch a `tf.Variable` but not a `tf.Tensor`.

한 가지 일반적인 오류가 실수로 대체하는 것입니다 `tf.Variable` 로모그래퍼 `tf.Tensor` 를 사용하는 대신, `Variable.assign` 업데이트 할 `tf.Variable` . 다음은 예입니다.

In [ ]:
x = tf.Variable(2.0)

for epoch in range(2):
  with tf.GradientTape() as tape:
    y = x+1

  print(type(x).__name__, ":", tape.gradient(y, x))
  x = x + 1   # This should be `x.assign_add(1)`

### 2. TensorFlow 외부에서 계산을 했습니까?

계산이 TensorFlow를 종료하면 테이프는 그래디언트 경로를 기록 할 수 없습니다. 예를 들면 :

In [ ]:
x = tf.Variable([[1.0, 2.0],
                 [3.0, 4.0]], dtype=tf.float32)

with tf.GradientTape() as tape:
  x2 = x**2

  # This step is calculated with NumPy
  y = np.mean(x2, axis=0)

  # Like most ops, reduce_mean will cast the NumPy array to a constant tensor
  # using `tf.convert_to_tensor`.
  y = tf.reduce_mean(y, axis=0)

print(tape.gradient(y, x))

### 3. 정수 또는 문자열을 통해 그라디언트 수행

정수와 문자열은 구별 할 수 없습니다. 계산 경로가 이러한 데이터 유형을 사용하는 경우 그라디언트가 없습니다.

아무도 문자열을 구별 할 수있을 것이라고 기대하지 않지만, `dtype` 지정하지 않으면 실수로 `int` 상수 또는 변수를 생성하기 쉽습니다.

In [ ]:
# The x0 variable has an `int` dtype.
x = tf.Variable([[2, 2],
                 [2, 2]])

with tf.GradientTape() as tape:
  # The path to x1 is blocked by the `int` dtype here.
  y = tf.cast(x, tf.float32)
  y = tf.reduce_sum(x)

print(tape.gradient(y, x))

TensorFlow는 유형간에 자동으로 캐스트되지 않으므로 실제로는 누락 된 그라디언트 대신 유형 오류가 발생하는 경우가 많습니다.

### 5. 상태 저장 객체를 통해 그라디언트 가져 오기

상태는 그라디언트를 중지합니다. 상태 저장 개체에서 읽을 때 테이프는 현재 상태 만 볼 수 있으며 이로 이어지는 기록은 볼 수 없습니다.

`tf.Tensor` 는 불변입니다. 일단 생성 된 텐서는 변경할 수 없습니다. *값* 은 있지만 *상태는* 없습니다. 지금까지 논의 된 모든 작업은 상태 비 저장입니다. `tf.matmul` 의 출력은 입력에만 의존합니다.

`tf.Variable` 에는 내부 상태, 값이 있습니다. 변수를 사용하면 상태를 읽습니다. 변수에 대한 기울기를 계산하는 것은 정상이지만 변수의 상태는 기울기 계산이 더 멀리 돌아가는 것을 차단합니다. 예를 들면 :


In [ ]:
x0 = tf.Variable(3.0)
x1 = tf.Variable(0.0)

with tf.GradientTape() as tape:
  # Update x1 = x1 + x0.
  x1.assign_add(x0)
  # The tape starts recording from x1.
  y = x1**2   # y = (x1 + x0)**2

# This doesn't work.
print(tape.gradient(y, x0))   #dy/dx0 = 2*(x1 + x2)

마찬가지로 `tf.data.Dataset` 반복자와 `tf.queue` 는 상태 저장이며이를 통과하는 텐서의 모든 그라디언트를 중지합니다.

## 등록 된 그라디언트 없음

Саммф `tf.Operation`s are **registered as being non-differentiable** and will return `None`. Others have **no gradient registered**.

[tf.raw_ops](https://www.tensorflow.org/api_docs/python/tf/raw_ops) 페이지에는 그라디언트가 등록 된 하위 수준 작업이 표시됩니다.

그래디언트가 등록되지 않은 부동 연산을 통해 그래디언트를 가져 오려고하면 테이프에서 조용히 `None` 반환하는 대신 오류가 발생합니다. 이렇게하면 무언가 잘못되었음을 알 수 있습니다.

예를 들어 `tf.image.adjust_contrast` 함수는 그라디언트를 가질 수 있지만 그라디언트가 구현되지 않은 [raw_ops.AdjustContrastv2](https://www.tensorflow.org/api_docs/python/tf/raw_ops#.AdjustContrastv2) 를 래핑합니다.


In [ ]:
image = tf.Variable([[[0.5, 0.0, 0.0]]])
delta = tf.Variable(0.1)

with tf.GradientTape() as tape:
  new_image = tf.image.adjust_contrast(image, delta)

try:
  print(tape.gradient(new_image, [image, delta]))
  assert False   # This should not happen.
except LookupError as e:
  print(f'{type(e).__name__}: {e}')


이 작업을 통해 차별화해야하는 경우 그래디언트를 구현하고 등록 ( `tf.RegisterGradient` 사용)하거나 다른 작업을 사용하여 함수를 다시 구현해야합니다.

## 없음 대신 0

어떤 경우에는 연결되지 않은 그라디언트에 대해 `None` 대신 0을 얻는 것이 편리 할 것입니다. `unconnected_gradients` 인수를 사용하여 그라디언트가 연결되지 않은 경우 반환 할 항목을 결정할 수 있습니다.

In [ ]:
x = tf.Variable([2., 2.])
y = tf.Variable(3.)

with tf.GradientTape() as tape:
  z = y**2
print(tape.gradient(z, x, unconnected_gradients=tf.UnconnectedGradients.ZERO))